In [22]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [23]:
import ROOT as root

In [24]:
I_POS = np.array([i for i in np.arange(0.1, 1.4, 0.1)])

In [25]:
I_POS


array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3])

In [26]:
I_NEG = I_POS[::-1]

In [27]:
I_NEG


array([1.3, 1.2, 1.1, 1. , 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1])

In [28]:
import uproot
import uproot3

In [46]:
_data0 = uproot.open('../dati/dataRUN_20220524_09.57.00.902015_HALL.root')

In [47]:
_data0.keys()

['data_BPOS;2', 'data_BPOS;1', 'data_BNEG;2', 'data_BNEG;1']

In [48]:
BPOS = _data0['data_BPOS']
BNEG = _data0['data_BNEG']

In [49]:
BPOS.keys()

['ID', 'RUN', 'BUFSIZE', 'V_gen_lo', 'V_gen_hi', 'V_out_lo', 'V_out_hi']

In [50]:
V_out_std = np.array([BNEG['V_gen_hi'].array(library='np'), 
                      BPOS['V_gen_hi'].array(library='np')]).std()
V_out_std

0.0025601029582906316

In [51]:
V_out = np.array([BNEG['V_gen_hi'].array(library='np'), 
                  BPOS['V_gen_hi'].array(library='np')]).mean()

V_out


4.998965462048848

In [52]:
import setup_hall.linear_regression as lg

i10mA = lg.linear_regression('generatore.txt', system='current')

Applying ATLAS style settings


Warning in <TCanvas::Constructor>: Deleting canvas with same name: __figure__


In [53]:
i10mA.set_model_parameters(1000/500, 0)
p0,p1 = i10mA.fit_model()

model results chi2/ndf (prob): 0.2796881563157294/9 (0.9999975624782381)
correlation parameter: 1.7242(300) [arb. u.]
offset: 0.0673(889) [coherent units with y-axis value]
 FCN=0.279688 FROM MIGRAD    STATUS=CONVERGED      50 CALLS          51 TOTAL
                     EDM=7.59718e-13    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.72418e+00   3.00379e-02   8.48703e-06  -6.62704e-05
   2  p1           6.73130e-02   8.89182e-02   2.51236e-05  -1.14248e-05


In [54]:
p0

(1.7241819129014446, 0.030037918506097314)

In [55]:
V_out*p0[0]/1000

0.008619125832883635

In [56]:
from uncertainties import ufloat

V_out_u = ufloat(V_out, V_out_std)
gain_gen = ufloat(p0[0], p0[1])/1000
current = V_out*gain_gen

In [57]:
current.std_dev/current.nominal_value

0.017421548318848592

In [58]:
(current*1000).format('.3uS')

'8.619(150)'

In [59]:
import awkward as ak 
import pandas as pd

ak_BPOS = _data0['data_BPOS'].arrays()
ak_BNEG = _data0['data_BNEG'].arrays()


In [60]:
print(ak.to_pandas(ak_BPOS))

                ID  RUN  BUFSIZE  V_gen_lo  V_gen_hi  V_out_lo  V_out_hi
entry subentry                                                          
0     0          1    1      100       0.0       5.0       0.0  0.366569
      1          1    1      100       0.0       5.0       0.0  0.376344
      2          1    1      100       0.0       5.0       0.0  0.366569
      3          1    1      100       0.0       5.0       0.0  0.376344
      4          1    1      100       0.0       5.0       0.0  0.371457
...             ..  ...      ...       ...       ...       ...       ...
11    95         1   12      100       0.0       5.0       0.0  0.322581
      96         1   12      100       0.0       5.0       0.0  0.327468
      97         1   12      100       0.0       5.0       0.0  0.322581
      98         1   12      100       0.0       5.0       0.0  0.312805
      99         1   12      100       0.0       5.0       0.0  0.322581

[1200 rows x 7 columns]
